# seq2seq & attention
- 튜토리얼에는 eng-fra 쌍을 가지고 진행했지만, 번역 결과물의 검증을 위해 end-ita 쌍을 전처리하여 활용함

## eng-ita 전처리

In [68]:
from tqdm import tqdm

In [69]:
# tab-seperated text이므로, 두 언어의 번역 쌍인 두 번째 tab까지의 데이터만 추림

text_file_path = './data/eng-ita.txt'
new_text_content = ''
with open(text_file_path, 'r', encoding='UTF-8') as f:
    lines = f.readlines()
    for i in tqdm(range(len(lines))):
        first_tab = lines[i].index('\t')
        ft_removed_lines = lines[i][first_tab+1:]
        second_tab = ft_removed_lines.index('\t')
        line = lines[i][:first_tab+second_tab+1]
        line = line + "\n"
        new_text_content += line
with open(text_file_path, 'w', encoding='UTF-8') as f:
    f.write(new_text_content)

100%|██████████| 354238/354238 [00:45<00:00, 7729.09it/s] 


## 데이터 전처리

In [70]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cpu


In [76]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {} # 
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # SOS 와 EOS 포함

    # 문장을 입력받아 단어로 쪼갬
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    # 입력받은 문장의 단어들을 인덱싱, 빈도 수 계산, 전체 단어 수 카운트함
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# 유니코드 문자열을 ASCII로 변환
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자로 변환, 알파벳이 아닌 문자 제거
# 알파벳을 사용하는 언어가 아닐 경우 추가 작업 필요
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [78]:
# 반대로 학습하고 싶다면 reverse=True (ita->eng)
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # 파일을 읽고 줄로 분리
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # 모든 줄을 쌍으로 분리하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # 쌍을 뒤집고, Lang 인스턴스 생성
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [73]:
# 최대 문장 길이 제한 및 어포스트로피 대체
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [83]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'ita', True)
print(random.choice(pairs))

Reading lines...
Read 354238 sentence pairs
Trimmed to 31599 sentence pairs
Counting words...
Counted words:
ita 5363
eng 3163
['io ne sono molto compiaciuta .', 'i m very pleased with it .']
